## CRF Model using Cross Validation

In [9]:
import pandas as pd
import sklearn_crfsuite
import nltk
from nltk.tokenize import word_tokenize

In [10]:
df= pd.read_csv('result.csv')
result = pd.DataFrame(df)
result.tokenized_text = result.tokenized_text.apply(lambda x: x.encode('ascii','ignore'))

In [11]:
x = []
y = []

for row in range(len(result)):
    if pd.notna(result['treated_text'][row]):
        text_iob = result['tokenized_text'][row].decode("utf-8").replace("', '", '').replace("[' ", '').replace("']", '')
        x.append(word_tokenize(result['treated_text'][row]))
        y.append(text_iob.split())
len(x), len(y)

(1168, 1168)

In [12]:
# Tirando os rotulos que nao batem com os dados
continua = True
while continua == True:
    continua = False
    for i in range(len(x)):
        if len(x[i]) > len(y[i]):
            x[i].pop()
            continua = True

        if len(x[i]) < len(y[i]):
            y[i].pop()
            continua = True

In [13]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            # print(sentence[i])
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

In [14]:
import math

split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(y_train)


(934, 234, 934)

In [15]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)

crf.fit(x_train, y_train)


/home/thais/Documentos/Knedle/venv/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [16]:
# from sklearn.preprocessing import MultiLabelBinarizer
# from sklearn.model_selection import cross_val_score

# mb = MultiLabelBinarizer()
# y_train = mb.fit_transform(y_train)

# recall = cross_val_score(crf, x_train, y_train, cv=5, scoring='recall')
# precision = cross_val_score(crf, x_train, y_train, cv=5, scoring='precision')
# accuracy = cross_val_score(crf, x_train, y_train, cv=5, scoring='accuracy')
# f1_score = cross_val_score(crf, x_train, y_train, cv=5, scoring='f1_macro')

# print(f"Recall: {recall}")
# print(f"Precision: {precision}")
# print(f"Acuracy: {accuracy}")
# print(f"f1_score: {f1_score}")

Recall: [nan nan nan nan nan]
Precision: [nan nan nan nan nan]
Acuracy: [nan nan nan nan nan]
f1_score: [nan nan nan nan nan]


/home/thais/Documentos/Knedle/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/thais/Documentos/Knedle/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/thais/Documentos/Knedle/venv/lib/python3.8/site-packages/sklearn_crfsuite/estimator.py", line 314, in fit
    trainer.append(xseq, yseq)
  File "pycrfsuite/_pycrfsuite.pyx", line 312, in pycrfsuite._pycrfsuite.BaseTrainer.append
  File "stringsource", line 48, in vector.from_py.__pyx_convert_vector_from_py_std_3a__3a_string
  File "stringsource", line 15, in string.from_py.__pyx_convert_string_from_py_std__in_string
TypeError: expected bytes, numpy.int64 found

  warnings.warn("Estimator fit failed. The score on this t

In [ ]:
classes = list(crf.classes_)
classes.remove('O')
print(classes)
y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

In [ ]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3
))